# 1. File Loading

In [140]:
import torch
import pandas as pd
import numpy as np
import os

# load data
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

# train
from torch import nn
from torch.nn import functional as F

# visualization
import matplotlib.pyplot as plt


In [163]:
batch_size = 32
num_workers = 40
train_set = 'train_set1.csv'
test_set = 'test_set1.csv'

In [166]:
class MFCCDataset(torch.utils.data.Dataset):
    def __init__(self, csv_file):
        self.full = pd.read_csv(csv_file, header=None, sep=r'\s*,\s*')
        self.label = torch.IntTensor(np.array(self.full[0].values).reshape(1922, 1))
        self.len = 1922
        self.data = torch.Tensor(np.array(self.full.loc[:,1:]).reshape(1922,100,1000))

    def __len__(self):
        return self.len

    def __getitem__(self, idx):
        return self.data[idx], self.label[idx]

print("Loading data...")

datasett = MFCCDataset(train_set)

dataloader = torch.utils.data.DataLoader(
   datasett, num_workers = num_workers
)
print("data is ready!")

Loading data...
data is ready!


In [170]:
dataloader

In [ ]:
datas = []
for i in range(1, len(dataset.loc[1,1:])):
    datas.append(dataset.loc[i,1:].values.reshape(100,1000))


# y = []
# x = []

# for data in dataset:
#     y.append(torch.Tensor([int(data[0])]))
#     x.append(torch.Tensor([data[1:]]))
    

# train_set = Data()

# for data in dataset:
#     train_set.add(data)

# print("dataset loaded")

# torch.utils.data.DataLoader(dataset, batch_size=1, shuffle=False, sampler=None,
# batch_sampler=None, num_workers=0, collate_fn=None, pin_memory=False, drop_last=False,
# timeout=0, worker_init_fn=None, multiprocessing_context=None )

In [ ]:
dataset.loc[i,1:].values.reshape(100,1000)[0]

In [142]:
datatest = np.array(dataset.loc[:,1:100000]).reshape(1922, 100,1000)

In [ ]:
print(datatest[0][0])
print(dataset.loc[0, 1:])


In [ ]:
import librosa
import librosa.display


In [ ]:
for i in range(100):
    plt.figure()
    librosa.display.specshow(datatest[i], sr=44100, hop_length = 1024)
    plt.show()

In [ ]:
import torch
import torch.nn as nn

inputs = torch.Tensor(1, 1, 100, 1000)
print('텐서의 크기 : {}'.format(inputs.shape))

In [ ]:
conv1 = nn.Conv2d(1, 32, 3, padding=1)
print(conv1)
conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
print(conv2)
pool = nn.MaxPool2d(2)
print(pool)

In [ ]:
out = conv1(inputs)
print(out.shape)

out = pool(out)
print(out.shape)

out = conv2(out)
print(out.shape)

out = pool(out)
print(out.shape)

out = out.view(out.size(0), -1) 
print(out.shape)

fc = nn.Linear(400000, 2)
out = fc(out)
print(out.shape)

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# 랜덤 시드 고정
torch.manual_seed(777)

# GPU 사용 가능일 경우 랜덤 시드 고정
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

learning_rate = 0.001
training_epochs = 15
batch_size = 100

data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)